In [6]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [8]:
subject = 'Amazonas - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [9]:
data = pd.read_csv('2003_01_model_input_AM.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego,Amazonas - Consumo de Cimento (t)
0,2003,623.227503,0.695898,3.993873,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,...,28.062332,23.292129,76.336632,1.379059,4.754469e+07,2.519796e+06,12.490621,4.318518e+07,8.630942,418.347000
1,2004,631.482259,0.697632,4.187699,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,...,28.186565,23.202276,75.477482,1.352970,4.823122e+07,2.538092e+06,12.509135,4.338144e+07,8.558110,587.382000
2,2005,637.838656,0.699241,4.449583,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,...,28.174554,23.394075,76.527436,1.251668,4.883625e+07,2.540251e+06,12.511306,4.349664e+07,8.526727,613.286000
3,2006,664.588528,0.700728,4.788581,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,...,28.026036,23.529800,77.812879,1.146689,4.938283e+07,2.557702e+06,12.511871,4.357888e+07,8.446276,626.147000
4,2007,691.804806,0.702099,5.207764,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,...,28.653553,23.387788,74.702779,1.336160,4.980707e+07,2.573779e+06,12.484991,4.354651e+07,8.434407,666.931000
5,2008,694.929186,0.703356,5.596891,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,...,29.368351,23.012008,70.408467,1.658576,5.017229e+07,2.568805e+06,12.460706,4.349579e+07,8.396886,769.233000
6,2009,715.226516,0.704503,5.981096,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,...,28.853124,22.769544,71.758162,1.685334,5.047410e+07,2.574930e+06,12.433926,4.345535e+07,8.420659,744.252000
7,2010,753.125620,0.705545,6.348653,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,...,28.071885,22.643630,74.482804,1.537130,5.169307e+07,2.569006e+06,12.457785,4.379523e+07,8.389077,796.116000
8,2011,768.056058,0.706485,6.656378,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,...,27.828830,22.427653,74.624578,1.468307,5.272696e+07,2.545977e+06,12.460974,4.398860e+07,8.376520,948.011000
9,2012,750.564710,0.707029,6.998333,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,...,27.699098,22.391859,74.970536,1.457821,5.353767e+07,2.524602e+06,12.414889,4.403561e+07,11.321769,994.932000


In [10]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego
0,623.227503,0.695898,3.993873,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,1418.883201,28.062332,23.292129,76.336632,1.379059,4.754469e+07,2.519796e+06,12.490621,4.318518e+07,8.630942
1,631.482259,0.697632,4.187699,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,1393.494298,28.186565,23.202276,75.477482,1.352970,4.823122e+07,2.538092e+06,12.509135,4.338144e+07,8.558110
2,637.838656,0.699241,4.449583,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,1384.531660,28.174554,23.394075,76.527436,1.251668,4.883625e+07,2.540251e+06,12.511306,4.349664e+07,8.526727
3,664.588528,0.700728,4.788581,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,1375.345978,28.026036,23.529800,77.812879,1.146689,4.938283e+07,2.557702e+06,12.511871,4.357888e+07,8.446276
4,691.804806,0.702099,5.207764,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,1380.313467,28.653553,23.387788,74.702779,1.336160,4.980707e+07,2.573779e+06,12.484991,4.354651e+07,8.434407
5,694.929186,0.703356,5.596891,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,1407.801557,29.368351,23.012008,70.408467,1.658576,5.017229e+07,2.568805e+06,12.460706,4.349579e+07,8.396886
6,715.226516,0.704503,5.981096,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,1393.526887,28.853124,22.769544,71.758162,1.685334,5.047410e+07,2.574930e+06,12.433926,4.345535e+07,8.420659
7,753.125620,0.705545,6.348653,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,1311.661141,28.071885,22.643630,74.482804,1.537130,5.169307e+07,2.569006e+06,12.457785,4.379523e+07,8.389077
8,768.056058,0.706485,6.656378,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,1276.962232,27.828830,22.427653,74.624578,1.468307,5.272696e+07,2.545977e+06,12.460974,4.398860e+07,8.376520
9,750.564710,0.707029,6.998333,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,1269.809720,27.699098,22.391859,74.970536,1.457821,5.353767e+07,2.524602e+06,12.414889,4.403561e+07,11.321769


In [11]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      587.382000
1      613.286000
2      626.147000
3      666.931000
4      769.233000
5      744.252000
6      796.116000
7      948.011000
8      994.932000
9     1036.606000
10     771.567857
11     746.867486
12     739.457657
13     687.211000
14     350.177000
15     533.444000
16     595.320000
17     610.155000
18     640.274000
19            NaN
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [12]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego
0,-0.064363,-1.902692,-1.871291,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,...,1.397476,-0.168944,1.254939,1.183825,-0.423813,-1.657429,0.147053,1.022575,-1.837871,-0.844534
1,0.015089,-1.569079,-1.691623,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,...,1.073077,0.121449,1.043768,0.678683,-0.623734,-1.420136,0.502943,1.329420,-1.291529,-0.879556
2,0.076268,-1.259532,-1.448868,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,...,0.958560,0.093375,1.494528,1.296009,-1.400021,-1.211011,0.544940,1.365402,-0.970828,-0.894648
3,0.333733,-0.973305,-1.134632,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,...,0.841193,-0.253786,1.813504,2.051791,-2.204482,-1.022092,0.884383,1.374762,-0.741870,-0.933333
4,0.595687,-0.709650,-0.746067,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,...,0.904663,1.213029,1.479753,0.223193,-0.752545,-0.875457,1.197126,0.929263,-0.831994,-0.939040
5,0.625759,-0.467820,-0.385363,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,...,1.255884,2.883864,0.596608,-2.301669,1.718150,-0.749223,1.100359,0.526764,-0.973198,-0.957083
6,0.821119,-0.247067,-0.029222,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,...,1.073494,1.679527,0.026778,-1.508109,1.923201,-0.644904,1.219506,0.082911,-1.085772,-0.945651
7,1.185894,-0.046644,0.311488,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,...,0.027480,-0.146614,-0.269140,0.093857,0.787503,-0.223579,1.104278,0.478356,-0.139598,-0.960838
8,1.329598,0.134197,0.596735,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,...,-0.415874,-0.714753,-0.776721,0.177214,0.260106,0.133776,0.656315,0.531211,0.398719,-0.966876
9,1.161246,0.238977,0.913712,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,...,-0.507263,-1.018001,-0.860844,0.380622,0.179750,0.413989,0.240530,-0.232601,0.529576,0.449384


In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      587.382000
1      613.286000
2      626.147000
3      666.931000
4      769.233000
5      744.252000
6      796.116000
7      948.011000
8      994.932000
9     1036.606000
10     771.567857
11     746.867486
12     739.457657
13     687.211000
14     350.177000
15     533.444000
16     595.320000
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [14]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego
17,-1.367217,0.772056,1.730097,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-2.182816,-1.2338,-0.810538,0.422015,-0.908652,0.870475,-0.673546,-0.73585,0.899019,0.934453


In [15]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    610.155
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [16]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [17]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [19]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2688844692, 3711696860, 3770545378, 3515777645, 1745599744, 2628221445, 1087389864, 1658671795, 2390556850, 2255721375, 2857594165, 2690257329, 1857536642, 3311797506, 1729373275, 520883799, 2105619043, 2175048106, 3241171880, 2398368133, 3208581192, 2115380626, 822518846, 1053396576, 3065482101, 2247388039, 2884190773, 2589672018, 1937238125, 588077911, 3458244465, 348955615, 2664133456, 1200086934, 2160060867, 979505968, 2795670996, 552285258, 3261218949, 1420121820, 2141901822, 3449525438, 3690306587, 1409844579, 2831955278, 1319711410, 2209415254, 3136715220, 1485453544, 3902040820, 1023502719, 2680874043, 1473346790, 388107389, 3699851104, 1963848199, 3486722210, 1351312234, 3726475898, 4003900183, 4226390526, 3262733703, 2743040264, 3388239158, 1088182090, 325974392, 811900140, 860273933, 4263433280, 691880312, 606570904, 3570367852, 4045966863, 3957898477, 1387855589, 2306777696, 1432133233, 4107444172, 4255874180, 4212206522, 3481047405, 3006033225, 1618302223, 710361332, 9384

val_loss: 15723.1025390625


Step: 87 ___________________________________________
val_loss: 42921.36328125


Step: 88 ___________________________________________
val_loss: 17764.69140625


Step: 89 ___________________________________________
val_loss: 11811.91796875


Step: 90 ___________________________________________
val_loss: 27154.896484375


Step: 91 ___________________________________________
val_loss: 28387.521484375


Step: 92 ___________________________________________
val_loss: 18884.453125


Step: 93 ___________________________________________
val_loss: 32630.966796875


Step: 94 ___________________________________________
val_loss: 21259.34765625


Step: 95 ___________________________________________
val_loss: 13779.0439453125


Step: 96 ___________________________________________
val_loss: 37425.99609375


Step: 97 ___________________________________________
val_loss: 22155.5390625


Step: 98 ___________________________________________
val_loss: 21580.619140625


Step: 99

In [20]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 470ms/step - loss: 543154.7500 - val_loss: 234637.1719
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 516670.8750 - val_loss: 39768.9727
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 355398.3125 - val_loss: 588481.1875
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 439975.8750 - val_loss: 286182.0312
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 423106.8438 - val_loss: 52751.6133
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 358205.8125 - val_loss: 15843.2119
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 377038.8438 - val_loss: 47879.5430
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 354218.3438 - val_loss: 40929.4062
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 353197.9375 - val_loss: 45909.8125
Epoch 10/10000


1/1 [==============================] - 0s 15ms/step - loss: 132162.3281 - val_loss: 13062.9922
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 130245.4844 - val_loss: 13096.0352
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 128370.5156 - val_loss: 12782.1299
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 126520.8125 - val_loss: 12697.7109
Epoch 79/10000
1/1 [==============================] - 0s 15ms/step - loss: 124704.4688 - val_loss: 12651.6748
Epoch 80/10000
1/1 [==============================] - 0s 15ms/step - loss: 121867.6953 - val_loss: 12278.2842
Epoch 81/10000
1/1 [==============================] - 0s 15ms/step - loss: 121206.3359 - val_loss: 12438.0127
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 119487.0312 - val_loss: 12235.8994
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 117401.6250 - val_loss: 12248.9736
Epoch 84/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 56212.6875 - val_loss: 14929.0908
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 56389.8164 - val_loss: 14821.3975
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 55152.0742 - val_loss: 15222.2734
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 54839.1328 - val_loss: 14928.3643
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 53952.1797 - val_loss: 15427.3721
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 53718.7070 - val_loss: 15358.3154
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 53157.3711 - val_loss: 15504.7705
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 52319.3203 - val_loss: 15805.8047
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 51748.6641 - val_loss: 15804.2891
Epoch 158/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 39951.1562 - val_loss: 37671.0430
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 37995.0742 - val_loss: 37745.2031
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 39502.7188 - val_loss: 37725.2227
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 39140.5234 - val_loss: 37255.3203
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 39078.4922 - val_loss: 38322.6875
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 38926.9375 - val_loss: 37817.6641
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 38914.0586 - val_loss: 38643.4297
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 38476.2031 - val_loss: 38451.6172
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 38277.4961 - val_loss: 38727.6680
Epoch 232/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 30581.8281 - val_loss: 49267.2617
Epoch 298/10000
1/1 [==============================] - 0s 15ms/step - loss: 29653.6953 - val_loss: 48366.0977
Epoch 299/10000
1/1 [==============================] - 0s 15ms/step - loss: 28929.0332 - val_loss: 47589.6758
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 30591.2969 - val_loss: 49304.3867
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 29693.4688 - val_loss: 48437.9531
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 28726.3262 - val_loss: 47933.1211
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 29354.2051 - val_loss: 49195.6914
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 30173.7480 - val_loss: 50012.5117
Epoch 305/10000
1/1 [==============================] - 0s 14ms/step - loss: 29234.9746 - val_loss: 48934.8125
Epoch 306/10000
1/1 [=====

...

1/1 [==============================] - 0s 14ms/step - loss: 11377.7725 - val_loss: 26263.5938
Epoch 742/10000
1/1 [==============================] - 0s 14ms/step - loss: 12096.9512 - val_loss: 24900.2734
Epoch 743/10000
1/1 [==============================] - 0s 14ms/step - loss: 11209.2646 - val_loss: 26544.8750
Epoch 744/10000
1/1 [==============================] - 0s 14ms/step - loss: 10531.8271 - val_loss: 24384.6074
Epoch 745/10000
1/1 [==============================] - 0s 14ms/step - loss: 10466.6680 - val_loss: 24707.5020
Epoch 746/10000
1/1 [==============================] - 0s 14ms/step - loss: 10389.2354 - val_loss: 26361.2422
Epoch 747/10000
1/1 [==============================] - 0s 14ms/step - loss: 10460.1240 - val_loss: 24215.9297
Epoch 748/10000
1/1 [==============================] - 0s 15ms/step - loss: 10945.7715 - val_loss: 27391.2891
Epoch 749/10000
1/1 [==============================] - 0s 15ms/step - loss: 10505.1387 - val_loss: 23824.6191
Epoch 750/10000
1/1 [=====

Epoch 816/10000
1/1 [==============================] - 0s 14ms/step - loss: 12753.4561 - val_loss: 30449.0879
Epoch 817/10000
1/1 [==============================] - 0s 14ms/step - loss: 13258.8711 - val_loss: 32526.9766
Epoch 818/10000
1/1 [==============================] - 0s 14ms/step - loss: 13841.4092 - val_loss: 34864.4492
Epoch 819/10000
1/1 [==============================] - 0s 15ms/step - loss: 12824.7549 - val_loss: 29354.3223
Epoch 820/10000
1/1 [==============================] - 0s 14ms/step - loss: 13595.3047 - val_loss: 32972.2617
Epoch 821/10000
1/1 [==============================] - 0s 14ms/step - loss: 12620.4316 - val_loss: 29699.9082
Epoch 822/10000
1/1 [==============================] - 0s 14ms/step - loss: 12512.0068 - val_loss: 30267.1641
Epoch 823/10000
1/1 [==============================] - 0s 14ms/step - loss: 12838.7295 - val_loss: 30180.9688
Epoch 824/10000
1/1 [==============================] - 0s 14ms/step - loss: 13231.9453 - val_loss: 32879.4727
Epoch 825/

1/1 [==============================] - 0s 14ms/step - loss: 13644.9922 - val_loss: 26420.8340
Epoch 891/10000
1/1 [==============================] - 0s 14ms/step - loss: 12919.2480 - val_loss: 30298.5391
Epoch 892/10000
1/1 [==============================] - 0s 14ms/step - loss: 14680.5811 - val_loss: 28529.3984
Epoch 893/10000
1/1 [==============================] - 0s 14ms/step - loss: 13715.3359 - val_loss: 35305.0820
Epoch 894/10000
1/1 [==============================] - 0s 14ms/step - loss: 13413.0859 - val_loss: 34165.7852
Epoch 895/10000
1/1 [==============================] - 0s 14ms/step - loss: 12879.2549 - val_loss: 29929.5000
Epoch 896/10000
1/1 [==============================] - 0s 14ms/step - loss: 13022.1611 - val_loss: 29671.8145
Epoch 897/10000
1/1 [==============================] - 0s 14ms/step - loss: 14844.7744 - val_loss: 30388.3594
Epoch 898/10000
1/1 [==============================] - 0s 14ms/step - loss: 14352.1250 - val_loss: 29662.9277
Epoch 899/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 17265.1836 - val_loss: 24850.3848
Epoch 965/10000
1/1 [==============================] - 0s 14ms/step - loss: 13246.6738 - val_loss: 29639.4473
Epoch 966/10000
1/1 [==============================] - 0s 14ms/step - loss: 11051.0664 - val_loss: 31345.2090
Epoch 967/10000
1/1 [==============================] - 0s 14ms/step - loss: 11144.1426 - val_loss: 24300.0801
Epoch 968/10000
1/1 [==============================] - 0s 14ms/step - loss: 10900.9287 - val_loss: 24828.2812
Epoch 969/10000
1/1 [==============================] - 0s 14ms/step - loss: 10384.3418 - val_loss: 26341.1406
Epoch 970/10000
1/1 [==============================] - 0s 14ms/step - loss: 10304.3477 - val_loss: 24705.7051
Epoch 971/10000
1/1 [==============================] - 0s 14ms/step - loss: 10344.3008 - val_loss: 26382.9277
Epoch 972/10000
1/1 [==============================] - 0s 14ms/step - loss: 10797.3525 - val_loss: 27289.8184
Epoch 973/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 11753.7188 - val_loss: 35046.3164
Epoch 1039/10000
1/1 [==============================] - 0s 14ms/step - loss: 11290.3691 - val_loss: 29650.5488
Epoch 1040/10000
1/1 [==============================] - 0s 14ms/step - loss: 13039.2734 - val_loss: 29884.7422
Epoch 1041/10000
1/1 [==============================] - 0s 14ms/step - loss: 10385.4707 - val_loss: 28864.6172
Epoch 1042/10000
1/1 [==============================] - 0s 14ms/step - loss: 10801.9834 - val_loss: 28545.7891
Epoch 1043/10000
1/1 [==============================] - 0s 14ms/step - loss: 11306.4170 - val_loss: 30933.9199
Epoch 1044/10000
1/1 [==============================] - 0s 14ms/step - loss: 13528.1045 - val_loss: 29738.4746
Epoch 1045/10000
1/1 [==============================] - 0s 14ms/step - loss: 11272.4795 - val_loss: 27652.9902
Epoch 1046/10000
1/1 [==============================] - 0s 14ms/step - loss: 13416.2256 - val_loss: 29774.8301
Epoch 1047/10000
1

In [21]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 49ms/step
[610.155](test_target) - [[584.89026]](prediction) = 25.264741210937473


In [22]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [23]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego
0,-1.254290,-1.651948,-1.338730,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,...,1.182999,-0.795076,0.466156,0.707433,-0.219657,-1.598377,-1.869038,0.296993,-1.892022,1.914585
1,-1.060035,-1.103970,-1.097441,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,...,0.341408,-0.522383,0.163126,0.338695,-0.365142,-1.042201,-0.905748,0.977617,-0.681310,1.018176
2,-0.910453,-0.595523,-0.771428,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,...,0.044315,-0.548745,0.809969,0.789324,-0.930057,-0.552049,-0.792075,1.057430,0.029372,0.631905
3,-0.280963,-0.125379,-0.349420,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,...,-0.260172,-0.874746,1.267701,1.341021,-1.515474,-0.109253,0.126701,1.078190,0.536750,-0.358282
4,0.359504,0.307689,0.172411,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,...,-0.095510,0.502665,0.788767,0.006201,-0.458880,0.234434,0.973204,0.090015,0.337034,-0.504360
5,0.433028,0.704909,0.656824,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,...,0.815665,2.071662,-0.478552,-1.836869,1.339078,0.530305,0.711284,-0.802782,0.024121,-0.966173
6,0.910675,1.067508,1.135111,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,...,0.342489,0.940730,-1.296261,-1.257595,1.488297,0.774812,1.033781,-1.787306,-0.225348,-0.673571
7,1.802535,1.396714,1.592673,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,...,-2.371196,-0.774106,-1.720905,-0.088210,0.661834,1.762328,0.721891,-0.910157,1.871404,-1.062282


train_target:


,Amazonas - Consumo de Cimento (t)
0,587.382
1,613.286
2,626.147
3,666.931
4,769.233
5,744.252
6,796.116
7,948.011


test_input:


...

,Amazonas - Produção de Cimento (t),Amazonas - IDH,Amazonas - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Desemprego
17,-1.367217,0.772056,1.730097,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-2.182816,-1.2338,-0.810538,0.422015,-0.908652,0.870475,-0.673546,-0.73585,0.899019,0.934453


test_target:


,Amazonas - Consumo de Cimento (t)
17,610.155


1/1 [==============================] - 0s 33ms/step
Error: 25.264741210937473




[839.8316650390625,
 892.2704467773438,
 1013.2550048828125,
 907.1920776367188,
 870.2857666015625,
 811.3712768554688,
 824.447021484375,
 744.7269897460938,
 634.5035400390625,
 584.8902587890625]

In [24]:
display(targets)
display(predictions)

[994.932,
 1036.606,
 771.5678571428572,
 746.8674857142857,
 739.4576571428571,
 687.211,
 350.177,
 533.444,
 595.32,
 610.155]

[839.8316650390625,
 892.2704467773438,
 1013.2550048828125,
 907.1920776367188,
 870.2857666015625,
 811.3712768554688,
 824.447021484375,
 744.7269897460938,
 634.5035400390625,
 584.8902587890625]

In [25]:
mae = mean_absolute_error(predictions, targets)
mae

170.6437306640625

In [26]:
porcentage = mae/np.mean(targets)
porcentage

0.24150871524540324